In [15]:
import numpy as np
import pandas as pd
from datetime import datetime

In [16]:
weather_df = pd.read_csv('weather.csv')
cab_weather_df = pd.read_csv('cab_rides.csv')

In [17]:
cab_weather_df.isnull().sum()

distance                0
cab_type                0
time_stamp              0
destination             0
source                  0
price               55095
surge_multiplier        0
id                      0
product_id              0
name                    0
dtype: int64

In [18]:
#Dropping instances where the price is null 
cab_weather_df.dropna(inplace=True)

cab_weather_df.isnull().sum()

distance            0
cab_type            0
time_stamp          0
destination         0
source              0
price               0
surge_multiplier    0
id                  0
product_id          0
name                0
dtype: int64

In [19]:
cab_weather_df.columns

Index(['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name'],
      dtype='object')

In [20]:
cab_weather_df = cab_weather_df[['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'name']]

cab_weather_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared
1,0.44,Lyft,1543284023677,North Station,Haymarket Square,11.0,1.0,Lux
2,0.44,Lyft,1543366822198,North Station,Haymarket Square,7.0,1.0,Lyft
3,0.44,Lyft,1543553582749,North Station,Haymarket Square,26.0,1.0,Lux Black XL
4,0.44,Lyft,1543463360223,North Station,Haymarket Square,9.0,1.0,Lyft XL


In [69]:
weather_df.head(30)

,temp,source,clouds,pressure,rain,time_stamp,humidity,wind,date,time
0,42.42,Back Bay,1.00,1012.14,0.1228,1545003901,0.77,11.25,2018-12-17,00:45:01
1,42.43,Beacon Hill,1.00,1012.15,0.1846,1545003901,0.76,11.32,2018-12-17,00:45:01
2,42.50,Boston University,1.00,1012.15,0.1089,1545003901,0.76,11.07,2018-12-17,00:45:01
3,42.11,Fenway,1.00,1012.13,0.0969,1545003901,0.77,11.09,2018-12-17,00:45:01
4,43.13,Financial District,1.00,1012.14,0.1786,1545003901,0.75,11.49,2018-12-17,00:45:01
5,42.34,Haymarket Square,1.00,1012.15,0.2068,1545003901,0.77,11.49,2018-12-17,00:45:01
6,42.36,North End,1.00,1012.15,0.2088,1545003901,0.77,11.46,2018-12-17,00:45:01
7,42.21,North Station,1.00,1012.16,0.2069,1545003901,0.77,11.37,2018-12-17,00:45:01
8,42.07,Northeastern University,1.00,1012.12,0.1020,1545003901,0.78,11.28,2018-12-17,00:45:01
9,43.05,South Station,1.00,1012.12,0.1547,1545003901,0.75,11.58,2018-12-17,00:45:01


In [21]:
weather_df.columns = ['temp', 'source', 'clouds', 'pressure', 'rain', 'time_stamp', 'humidity', 'wind']

In [58]:
#we want to convert the timestamps into date, time, weekday, hour, and time of day (Morning, afternoon, evening, and night)
cab_weather_df["rounded_timestamp"] = cab_weather_df["time_stamp"] / 1000
cab_weather_df["rounded_timestamp"] = cab_weather_df["rounded_timestamp"].apply(np.floor)

cab_weather_df["date"] = cab_weather_df["rounded_timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
cab_weather_df["time"] = cab_weather_df["rounded_timestamp"].apply(lambda x: datetime.fromtimestamp(x).time())
cab_weather_df['weekday'] = cab_weather_df['date'].apply(lambda x: x.weekday())
cab_weather_df["weekday"] = cab_weather_df["weekday"].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
cab_weather_df['hour'] = cab_weather_df['time'].apply(lambda time: time.hour)

cab_weather_df.loc[(cab_weather_df.hour >= 6) & (cab_weather_df.hour < 12) , 'time_of_day'] = 'Morning'
cab_weather_df.loc[(cab_weather_df.hour >= 12) & (cab_weather_df.hour < 15) , 'time_of_day'] = 'Afternoon'
cab_weather_df.loc[(cab_weather_df.hour >= 15) & (cab_weather_df.hour < 18) , 'time_of_day'] = 'Evening'
cab_weather_df.loc[(cab_weather_df.hour >= 18) | (cab_weather_df.hour < 6) , 'time_of_day'] = 'Night'

cab_weather_df = cab_weather_df[['distance', 'cab_type', 'time_stamp', 'destination', 'source', 'price',
       'surge_multiplier', 'name', 'date', 'time', 'weekday', 'hour', 'time_of_day']]

In [59]:
#we want to convert the timestamps into date, time, weekday, hour, and time of day (Morning, afternoon, evening, and night)
#weather_df["rounded_timestamp"] = weather_df["time_stamp"] / 1000
#weather_df["rounded_timestamp"] = weather_df["rounded_timestamp"].apply(np.floor)

weather_df["date"] = weather_df["time_stamp"].apply(lambda x : datetime.fromtimestamp(x).date())
weather_df["time"] = weather_df["time_stamp"].apply(lambda x: datetime.fromtimestamp(x).time())
#weather_df['weekday'] = weather_df['date'].apply(lambda x: x.weekday())
#weather_df["weekday"] = weather_df["weekday"].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
#weather_df['hour'] = weather_df['time'].apply(lambda time: time.hour)

#weather_df.loc[(weather_df.hour >= 6) & (weather_df.hour < 12) , 'time_of_day'] = 'Morning'
#weather_df.loc[(weather_df.hour >= 12) & (weather_df.hour < 15) , 'time_of_day'] = 'Afternoon'
#weather_df.loc[(weather_df.hour >= 15) & (weather_df.hour < 18) , 'time_of_day'] = 'Evening'
#weather_df.loc[(weather_df.hour >= 18) | (weather_df.hour < 6) , 'time_of_day'] = 'Night'

weather_df = weather_df[['temp', 'source', 'clouds', 'pressure', 'rain', 'time_stamp', 'humidity', 
                         'wind', 'date', 'time']]

In [67]:
merged_data = pd.merge(cab_weather_df, weather_df, how="outer", on=["date", "source"])

In [68]:
merged_data.head(30)

,distance,cab_type,time_stamp_x,destination,source,price,surge_multiplier,name,date,time_x,...,hour,time_of_day,temp,clouds,pressure,rain,time_stamp_y,humidity,wind,time_y
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,41.68,0.44,1023.68,NaN,1544924701,0.71,6.28,02:45:01
1,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,42.75,0.17,1023.04,NaN,1544921101,0.71,6.45,01:45:01
2,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,40.99,0.40,1024.49,NaN,1544928301,0.71,6.38,03:45:01
3,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,40.15,0.41,1023.50,NaN,1544939101,0.70,7.82,06:45:01
4,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,41.95,0.87,1021.58,NaN,1544971501,0.71,7.33,15:45:01
5,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,41.99,0.95,1013.20,0.0040,1544996701,0.77,10.71,22:45:01
6,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,40.62,0.55,1024.65,NaN,1544931901,0.71,6.61,04:45:01
7,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,42.35,0.91,1018.92,0.0080,1544978701,0.73,9.47,17:45:01
8,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,41.41,1.00,1022.15,NaN,1544967901,0.71,6.87,14:45:01
9,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,Shared,2018-12-16,10:30:07,...,10,Morning,42.49,1.00,1013.99,0.0030,1544993101,0.74,9.92,21:45:01


In [62]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24454238 entries, 0 to 24454237
Data columns (total 21 columns):
distance            float64
cab_type            object
time_stamp_x        int64
destination         object
source              object
price               float64
surge_multiplier    float64
name                object
date                object
time_x              object
weekday             object
hour                int64
time_of_day         object
temp                float64
clouds              float64
pressure            float64
rain                float64
time_stamp_y        int64
humidity            float64
wind                float64
time_y              object
dtypes: float64(9), int64(3), object(9)
memory usage: 4.0+ GB


In [33]:
merged_data.count()

temp                  6276
source              644252
clouds                6276
pressure              6276
rain                   894
time_stamp_x          6276
humidity              6276
wind                  6276
date                644252
time_x                6276
weekday_x             6276
hour_x                6276
time_of_day_x         6276
distance            637976
cab_type            637976
time_stamp_y        637976
destination         637976
price               637976
surge_multiplier    637976
name                637976
time_y              637976
weekday_y           637976
hour_y              637976
time_of_day_y       637976
dtype: int64

In [34]:
merged_data.isnull().sum()

temp                637976
source                   0
clouds              637976
pressure            637976
rain                643358
time_stamp_x        637976
humidity            637976
wind                637976
date                     0
time_x              637976
weekday_x           637976
hour_x              637976
time_of_day_x       637976
distance              6276
cab_type              6276
time_stamp_y          6276
destination           6276
price                 6276
surge_multiplier      6276
name                  6276
time_y                6276
weekday_y             6276
hour_y                6276
time_of_day_y         6276
dtype: int64

In [35]:
#merged_data.dropna(inplace=True)

merged_data.head()

,temp,source,clouds,pressure,rain,time_stamp_x,humidity,wind,date,time_x,...,cab_type,time_stamp_y,destination,price,surge_multiplier,name,time_y,weekday_y,hour_y,time_of_day_y
0,42.42,Back Bay,1.00,1012.14,0.1228,1.545004e+09,0.77,11.25,1970-01-18,22:10:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,43.28,Back Bay,0.81,990.81,NaN,1.543348e+09,0.71,8.30,1970-01-18,21:42:27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39.23,Back Bay,0.83,996.09,NaN,1.543450e+09,0.66,10.67,1970-01-18,21:44:09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27.27,Back Bay,0.15,1033.40,NaN,1.544788e+09,0.80,2.96,1970-01-18,22:06:27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45.49,Back Bay,1.00,1012.46,NaN,1.543254e+09,0.86,6.47,1970-01-18,21:40:53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
merged_data.head(5)

,temp,source,clouds,pressure,rain,time_stamp_x,humidity,wind,date,time_x,...,cab_type,time_stamp_y,destination,price,surge_multiplier,name,time_y,weekday_y,hour_y,time_of_day_y
